In [ ]:
%%bash
ls /sys/bus/usb-serial/devices/ | sed "s/^/w/dev\//g"

In [ ]:
import serial

In [ ]:
ser1 = serial.Serial('/dev/ttyUSB0', 9600)   #    Touch Sensor 
ser2 = serial.Serial('/dev/ttyUSB1', 9600)   #    Password Button
ser3 = serial.Serial('/dev/ttyUSB2', 9600)   #    Vibration Sensor

In [ ]:
import RPi.GPIO as GPIO 
import time 

#    [ Servo Setting ]
servoPin = 23                          
GPIO.setmode(GPIO.BCM)
GPIO.setup(servoPin, GPIO.OUT)
servo = GPIO.PWM(servoPin, 50)         #    50 Hz in channel servoPin
servo.start(2.5)                       #    servo initialization


#    [ Password Setting ]
password = [8, 1, 3,  1, 0]            #   Keypad Configuration
                                       #  { '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '*', '#' }
                
                                       #    '*' :  terminate input
                                       #    '#' :  delete last input


#    [ Buzzer Setting ]
buzzerPin = 12                         
scale = [ 261, 294, 329, 349, 392, 440, 493, 523, 587, 659, 698, 784 ]
#          도   레   미   파   솔   라   시   도   레   미   파   솔 
GPIO.setup(buzzerPin, GPIO.OUT)
buzzer = GPIO.PWM(buzzerPin, 100)
buzzer.ChangeDutyCycle(90)

beep = [ 9, 2, 9, 2, 9, 2, 9, 2 ]     
dingdong = [ 0, 2, 4, 7 ]             
error = [ 6, 6, 6 ]                   


#    [ Touch Sensor initialization ]
touch = True                        #    if Touch Sensor is on, initialize it by True

def buttonSound(num):
    buzzer.start(1)
    buzzer.ChangeFrequency(scale[num])
    time.sleep(0.3)
    buzzer.stop()
    
def noteSound(soundList):
    for i in range(len(soundList)):
        buzzer.start(1)
        buzzer.ChangeFrequency(scale[soundList[i]])
        time.sleep(1)
        buzzer.stop()
        
def rotateServo(num):
    servo.ChangeDutyCycle(num)
    time.sleep(1)
    servo.stop()

def toASCII(serialRead):
    if int(serialRead) == 57:
        return True
    else:
        return False

In [ ]:
# Step1. Touch Sensor

ser1.flushInput()

while True:
    try:
        tou = ser1.readline()
        print(int(tou[0]))
        
        if touch:
            if toASCII(tou[0]):
                print("        Problem Occurred !!!")
                noteSound(beep)
                break
        if not toASCII(tou[1]) and toASCII(tou[2]):
            print("        Fire Occurred !!!")
            noteSound(beep)
            break
        
    except KeyboardInterrupt:
        break

In [ ]:
# Step2. Password Button

ser2.flushInput()
test = []
password = [ 0, 1 ]   # you can change password

while True:
    try:
        while True:
            try:
                pas = ser2.readline()
                if pas[0] == 0:          
                        print(" ")
                else:
                    print(int(pas[0]) - 48)

      
                if pas[0] == 0:
                    break
                num = int(pas[0]) - 48
                if int(pas[0]) == 42:       
                    print(test)
                    print(password)
                    buttonSound(10)
                    
                    if password == test:    
                        noteSound(dingdong)
                        rotateServo(2.5)
                        print(test)
                        print("        The Safe Is Opened")
                        break
                    else:                   
                        noteSound(error)
                        print("        Wrong  !!!!")
                        break
                elif 0 <= num and num <= 9:
                    num = int(pas[0]) - 48
                    buttonSound(num)
                    print("Num {} has been pressed".format(num))
                    test.append(num)
                elif int(pas[0]) == 35:           
                    buttonSound(11)
                    del test[-1]
            except KeyboardInterrupt:
                break
            
    except KeyboardInterrupt:
        break

In [ ]:
# Step3. Vibration Sensor

ser3.flushInput() 
vib = ser3.readline()

while True:
    try:
        if not toASCII(vib[0]):                    
            print("        Vibration Occurred !!!")
            noteSound(beep)
            break
        print(vib[0])

    except KeyboardInterrupt:
        break